# Neural Network Model Testing

This notebook tests the neural network model for sentiment prediction on news headlines.

In [1]:
# Import necessary libraries
import sys
import os
import pandas as pd
from pathlib import Path

# Add project root to path for imports
sys.path.append(os.path.abspath('../..'))

# Import project modules
from src.models.predict_model import ModelPredictor
from src.models.train_model import ModelTrainer
from src.config import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
d:\Documents\CODE\HCMUT\Machine Learning Assignment\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Initialize the Model Predictor

We'll initialize the ModelPredictor class that will use our trained neural network model.

In [30]:
model_dir = Path(MODEL_DIR)
models = list(model_dir.glob('*rrnn_*.pkl'))
# if not models:
#     model_dir = Path(EXPERIMENT_DIR)
#     models = list(model_dir.glob('*rnn_*.pkl'))
if not models:
    trainer = ModelTrainer()
    trainer.train_neural_network(batch_size=2)

# Initialize the predictor
predictor = ModelPredictor()

Starting RNN model training...
Class weights for balancing: {0: np.float64(2.6749482401656315), 1: np.float64(0.5610073816760747), 2: np.float64(1.1853211009174311)}
Epoch 1/20
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 31s 16ms/step - accuracy: 0.4762 - loss: 1.1039 - val_accuracy: 0.6546 - val_loss: 0.8579 - learning_rate: 0.0010
Epoch 2/20
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 25s 16ms/step - accuracy: 0.6787 - loss: 0.8264 - val_accuracy: 0.6753 - val_loss: 0.7579 - learning_rate: 0.0010
Epoch 3/20
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 23s 15ms/step - accuracy: 0.8098 - loss: 0.5759 - val_accuracy: 0.7191 - val_loss: 0.7573 - learning_rate: 0.0010
Epoch 4/20
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 24s 16ms/step - accuracy: 0.8860 - loss: 0.3470 - val_accuracy: 0.7113 - val_loss: 0.8725 - learning_rate: 0.0010
Epoch 5/20
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.9301 - loss: 0.2496 - val_accuracy: 0.7332 - val_loss: 0.9755 - learning_rate: 0.0010
Epoch 6/20
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/ste

## 2. Test Single Headline Prediction

Let's test the model on a single headline first to check if everything is working.

In [31]:
# Test with a single positive headline
test_headline = "Company profits exceed expectations in Q1 2025"
result = predictor.predict_neural_network(test_headline)

# Display the result
if result:
    r = result[0]  # Get the first result
    print(f"\nHeadline: {r['headline']}")
    print(f"Predicted Sentiment: {r['sentiment']}")
    print(f"Confidence: {r['confidence']:.2f}")
    
    print("\nAll Probabilities:")
    for sentiment, prob in r['probabilities'].items():
        print(f"- {sentiment}: {prob:.2f}")
else:
    print("Prediction failed or no model found.")

Using most recent model: d:\Documents\CODE\HCMUT\Machine Learning Assignment\models\experiments\rnn_20250304_190812.pkl
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 514ms/step
Prediction class distribution:
negative (class 0): 0 predictions
neutral (class 1): 1 predictions
positive (class 2): 0 predictions

Headline: Company profits exceed expectations in Q1 2025
Predicted Sentiment: neutral
Confidence: 0.66

All Probabilities:
- negative: 0.12
- neutral: 0.66
- positive: 0.22


## 3. Test Multiple Headlines

Now let's test the model on multiple headlines with different expected sentiments.

In [32]:
# Test with multiple headlines
test_headlines = [
    "Stock market reaches all-time high as investor confidence grows",
    "Major company announces significant layoffs due to economic downturn",
    "Global trade continues at steady pace despite mild fluctuations",
    "Tech giant releases new product line with innovative features",
    "Retail sales decline for third consecutive quarter"
]

results = predictor.predict_neural_network(test_headlines)

# Display the results
if results:
    for r in results:
        print(f"Headline: {r['headline']}")
        print(f"Predicted Sentiment: {r['sentiment']} (confidence: {r['confidence']:.2f})")
        print()
else:
    print("Prediction failed or no model found.")

Using most recent model: d:\Documents\CODE\HCMUT\Machine Learning Assignment\models\experiments\rnn_20250304_190812.pkl
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 565ms/step
Prediction class distribution:
negative (class 0): 3 predictions
neutral (class 1): 0 predictions
positive (class 2): 2 predictions
Headline: Stock market reaches all-time high as investor confidence grows
Predicted Sentiment: negative (confidence: 0.43)

Headline: Major company announces significant layoffs due to economic downturn
Predicted Sentiment: positive (confidence: 0.28)

Headline: Global trade continues at steady pace despite mild fluctuations
Predicted Sentiment: negative (confidence: 0.44)

Headline: Tech giant releases new product line with innovative features
Predicted Sentiment: positive (confidence: 0.36)

Headline: Retail sales decline for third consecutive quarter
Predicted Sentiment: negative (confidence: 0.87)



## 4. Test on Real Dataset

Let's load a sample of the test dataset and predict sentiments.

In [33]:
# Load test dataset
test_data_path = os.path.join(RAW_DATA_PATH, "all-data.csv")
test_df = pd.read_csv(test_data_path)

print(f"Loaded test data with {len(test_df)} headlines")

# Show a few examples
test_df.head(100)


Loaded test data with 4846 headlines


,Sentiment,News Headline
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
95,positive,Finnish financial group Aktia reports operatin...
96,positive,Finnish high technology provider Vaahto Group ...
97,positive,Net sales of Finnish food industry company L+�...
98,positive,An individual promotion also generated slightl...


In [34]:
# Make predictions
headlines = test_df[' News Headline'].tolist()
results = predictor.predict_neural_network(headlines)

# Create a dataframe with predictions
if results:
    predicted_sentiments = [r['sentiment'] for r in results]
    confidence_scores = [round(r['confidence'], 2) for r in results]
    
    # Add predictions to the dataframe
    results_df = test_df.copy()
    results_df = results_df.rename(columns={'Sentiment': 'Actual Sentiment'})
    results_df['Predicted Sentiment'] = predicted_sentiments
    results_df['Confidence'] = confidence_scores
    
    print(f"Predictions completed")
else:
    print("Prediction failed or no model found.")

# Show some results
results_df.head(3)

Using most recent model: d:\Documents\CODE\HCMUT\Machine Learning Assignment\models\experiments\rnn_20250304_190812.pkl
152/152 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
Prediction class distribution:
negative (class 0): 1195 predictions
neutral (class 1): 2325 predictions
positive (class 2): 1326 predictions
Predictions completed


,Actual Sentiment,News Headline,Predicted Sentiment,Confidence
0,neutral,"According to Gran , the company has no plans t...",positive,0.41
1,neutral,Technopolis plans to develop in stages an area...,positive,0.25
2,negative,The international electronic industry company ...,negative,0.74


## 5. Model Evaluation

Let's evaluate the model's performance on the test dataset.

In [35]:
from sklearn.metrics import classification_report

# Calculate metrics
y_true = results_df['Actual Sentiment']
y_pred = results_df['Predicted Sentiment']

# Print classification report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

    negative       0.47      0.93      0.62       604
     neutral       0.93      0.75      0.83      2879
    positive       0.70      0.68      0.69      1363

    accuracy                           0.75      4846
   macro avg       0.70      0.79      0.71      4846
weighted avg       0.81      0.75      0.76      4846



In [36]:
true = results_df['Predicted Sentiment'] == results_df['Actual Sentiment']
true.value_counts()

True     3642
False    1204
Name: count, dtype: int64